In [1]:
#separate data into text and labels
import pandas as pd
import numpy as np

df = pd.read_csv('text.csv')

text = []
labels = []
for row in df.iterrows():
    text.append(row[1]['text'])
    labels.append(row[1]['label'])

labels = np.array(labels)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.file_utils import is_tf_available, is_torch_available
import torch
from sklearn.model_selection import train_test_split

In [ ]:
bert_v = "bert-base-uncased"

tokenizer = BertTokenizerFast.from_pretrained(bert_v, do_lower_case=True)

model = BertForSequenceClassification.from_pretrained(bert_v, num_labels=6)

In [ ]:
import random
def set_seed(seed: int): #function to ensure reproducable behaviour
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    if is_tf_available():
        import tensorflow as tf
        tf.random.set_seed(seed)

set_seed(1)

In [ ]:
#split text into training validation and test data

def train_val_test_split(data):
    return data[:round(0.6*len(data))], data[round(0.6*len(data)):round(0.8*len(data))], data[round(0.8*len(data)):]

train_text, val_text, test_text = train_val_test_split(text)


In [ ]:
train_tokens = tokenizer(train_text, truncation=True, padding=True, max_length=200)
val_tokens = tokenizer(val_text, truncation=True, padding=True, max_length=200)
test_tokens = tokenizer(test_text, truncation=True, padding=True, max_length=200)

In [ ]:
train_labels, val_labels, test_labels = labels[:round(0.6*len(labels))], labels[round(0.6*len(labels)):round(0.8*len(labels))], labels[round(0.8*len(labels)):]

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = Dataset(train_tokens, train_labels)
val_dataset = Dataset(val_tokens, val_labels)
test_dataset = Dataset(test_tokens, test_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=20,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    logging_steps=400,
    save_steps=400,
    evaluation_strategy="steps",
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return acc,f1

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

cant seem to run on locally, trying colab